<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Gathering Intelligence with Generative AI   

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.39"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.39-alpha"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info using REASONING o-series of models
var azureOpenAIEndpoint = config["AzureOpenAI:reasoningEndpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:reasoningAPIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:reasoningModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Json, 9.0.0 Microsoft.SemanticKernel, 1.39.0 Microsoft.SemanticKernel.Plugins.Core, 1.39.0-alpha

In [2]:
using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Using Azure OpenAI Service


### Step 2 - Gathering Intelligence with Built-In Generative AI Knowledge 

Generative AI models are trained on very large amounts of data. This data includes decision-making techniques, decision frameworks, reasoning techniques, decision best practices and other adjacent decision data that can help make decisions. Therefore, an obvious step in gathering Decision Intelligence that built-into the GenAI model is important to understand.  

Certain models (typically those that have more parameters) have more "intelligence built-in". Furthermore, models that specialize in certain tasks or a fine-tuned for a specific domain can also have more intelligence built-in. A great example of model specialization is the OpenAI o-series of models. The o-series of models are advanced reasoning models that are a great fit for decision-making. Therefore, it is a good hypotheses to test if these reasoning models have additional Decision Intelligence data built-in.  

Execute the cell below to view the prompt output asking for a "complete" list of Decision Frameworks built-into the o1 series of models.

In [4]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";
var decisionAnalysisPrompt = """
Identify and a very complete list of decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each framework supports better analysis and reasoning in different decision-making scenarios.

Create a column for the Number (in ascending order), framework name and decsription.
Surface the gathered frameworks in a Markdown table format. Do not surround the markdown with triple backticks.
Only return the Markdown table no other text.
""";

// How the prompt looks like to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: {systemPromptForDecisions}

Request from the user: {decisionAnalysisPrompt}
""";

#pragma warning disable SKEXP0010
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ReasoningEffort = "high"
};

KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);
var simplePromptResponse = await semanticKernel.InvokePromptAsync(decisionAnalysisPrompt, kernelArguments);
Console.WriteLine(simplePromptResponse.GetValue<string>());

| Number | Framework Name | Description |
| --- | --- | --- |
| 1 | SWOT Analysis | Analyzes internal strengths and weaknesses alongside external opportunities and threats, helping to clarify strategic directions. |
| 2 | PEST Analysis | Examines Political, Economic, Social, and Technological factors to assess external influences that affect decision outcomes. |
| 3 | Decision Matrix Analysis | Uses weighted criteria to systematically compare alternatives, ensuring a clear and objective evaluation of options. |
| 4 | Cost-Benefit Analysis | Weighs the expected costs against the potential benefits to identify the most economically advantageous option. |
| 5 | Risk Analysis | Identifies potential risks and their impacts, enabling decision-makers to manage uncertainties effectively. |
| 6 | Force Field Analysis | Maps driving and restraining forces in a situation, aiding in the development of strategies to overcome resistance. |
| 7 | Analytical Hierarchy Process (AHP) | Breaks complex de

| Number | Framework Name | Description |
| --- | --- | --- |
| 1 | SWOT Analysis | Analyzes internal strengths and weaknesses alongside external opportunities and threats, helping to clarify strategic directions. |
| 2 | PEST Analysis | Examines Political, Economic, Social, and Technological factors to assess external influences that affect decision outcomes. |
| 3 | Decision Matrix Analysis | Uses weighted criteria to systematically compare alternatives, ensuring a clear and objective evaluation of options. |
| 4 | Cost-Benefit Analysis | Weighs the expected costs against the potential benefits to identify the most economically advantageous option. |
| 5 | Risk Analysis | Identifies potential risks and their impacts, enabling decision-makers to manage uncertainties effectively. |
| 6 | Force Field Analysis | Maps driving and restraining forces in a situation, aiding in the development of strategies to overcome resistance. |
| 7 | Analytical Hierarchy Process (AHP) | Breaks complex decisions into a hierarchy of criteria and sub-criteria, employing pairwise comparisons to set priorities. |
| 8 | Monte Carlo Simulation | Uses probabilistic modeling to simulate various scenarios under uncertainty, offering insights into potential outcomes. |
| 9 | Decision Tree Analysis | Visualizes decision paths, associated probabilities, and payoffs, enabling a clear understanding of potential consequences. |
| 10 | Scenario Planning | Develops multiple future scenarios to test strategies against different potential environments, enhancing strategic flexibility. |
| 11 | Six Thinking Hats | Encourages thinking from diverse perspectives (e.g., emotional, analytical, creative) to ensure a holistic evaluation of issues. |
| 12 | Delphi Method | Gathers iterative expert insights to achieve consensus, reducing bias and enhancing decision quality in complex situations. |
| 13 | OODA Loop (Observe, Orient, Decide, Act) | Promotes rapid, iterative decision cycles to adapt quickly in dynamic, competitive, or high-pressure environments. |
| 14 | Vroom-Yetton Decision Model | Aligns decision-making processes with situational factors by balancing autocratic and participative approaches for optimal leadership decisions. |
| 15 | Eisenhower Matrix | Prioritizes tasks based on urgency and importance, guiding individuals to focus on high-impact activities. |
| 16 | Pareto Analysis | Identifies the critical few factors that yield the majority of results, streamlining focus and resource allocation. |
| 17 | PDCA Cycle (Plan-Do-Check-Act) | Implements continuous improvement through an iterative cycle of planning, execution, evaluation, and refinement. |
| 18 | Game Theory | Analyzes strategic interactions where outcomes depend on the actions of multiple agents, enhancing competitive decision-making. |
| 19 | Sensitivity Analysis | Tests how variations in key variables affect outcomes, ensuring decisions remain robust under different scenarios. |
| 20 | Failure Mode and Effects Analysis (FMEA) | Identifies and prioritizes potential failure points within a process to proactively mitigate risks and enhance reliability. |

The prompt paired with the reasoning model above likely returned a list of ~20 (depending on randomness) Decision Frameworks it is aware of. Presumably, because it described these decision frameworks, it should be able to operate on them to make decisions. This highlights just some of decision-making capability of the reasoning series of model. What if there are more decision frameworks that are available? This is potentially just a list of the top frameworks (and certainly not complete) of frameworks the model has identified. There could be a great deal more!

The code below will iterate several times (5x) and continue to prompt the GenAI model with the same instruction until a more comprehensive list is built. Notice the output below can have well over 100 decision-frameworks that the o1 series model is aware of.

In [5]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var markDownTaleOfDecisionFrameworks = string.Empty;

// This can quite a long time. View the output results below to see the final markdown table
// Note: This is done for illustrative purposes. 
// Ideally, the markdown formatting and combining of tables should be done in seperate steps for optimal results.
for (int i = 0; i != 5; i++)
{
    var systemPromptForDecisions = $"""
    You are a Decision Intelligence Assistant. 
    Assist the user in exploring options, reasoning through decisions, problem-solving.
    Apply systems thinking to various scenarios. 
    Provide structured, logical, and comprehensive advice.
    """;
    var decisionAnalysisPrompt = $"""
    Identify and a very comprehensive list of decision-making frameworks that can enhance the quality of decisions. 
    Briefly describe how each framework supports better analysis and reasoning in different decision-making scenarios.

    Surface the gathered frameworks in a Markdown table format. Do not surround the markdown with triple backticks.
    The FOUR table columns should be: 
    Number (in ascending order), Decision-Framework Name, 
    Value (a number between 1 to 10, how valueable the framework is) and Description.
    Only return the Markdown table no other text.

    This prompt is going to be called several times to gather a full list of decision-making frameworks. 
    The previous Markdown table is located in the area surrounded by <DecisionMakingFrameworks> tags.
    Make sure to copy the table found in the previous responses and append new unique frameworks to the table. 
    DO NOT discard the previous Markdown table found in the <DecisionMakingFrameworks> tags.
    Add new unique frameworks by appending to the previous Markdown table. Do not recreate the table header.
    The final Markdown table output should include both:
    The previous markdown table (found in the <DecisionMakingFrameworks> Tags) combined with new unique frameworks output in a single table.

    <DecisionMakingFrameworks>
    {markDownTaleOfDecisionFrameworks}
    </DecisionMakingFrameworks>.
    """;

    // How the prompt looks like to the LLM
    var simpleDecisionPromptTemplate = $"""
    System Prompt: {systemPromptForDecisions}

    Request from the user: {decisionAnalysisPrompt}
    """;

    // Uncomment this line to view the debugging 
    // Console.WriteLine(decisionAnalysisPrompt);

    var simplePromptResponse = await semanticKernel.InvokePromptAsync(decisionAnalysisPrompt);
    // This will append the new markdown table to the previous markdown tables
    markDownTaleOfDecisionFrameworks = simplePromptResponse.GetValue<string>();
    Console.WriteLine($"Finished Gethering Intelligence Iteration: {i + 1}");
}

Console.WriteLine(string.Empty);
Console.WriteLine("Filtering for RISK ANALYSIS frameworks...");

// Create a system prompt instruction to filter the markdown table for RISK ANALYSIS frameworks
var riskAnalysisFrameworksPrompt = $"""
You have the following Markdown table of decision-making frameworks.
The table is located in the area surrounded by <DecisionMakingFrameworks> tags.

<DecisionMakingFrameworks>
{markDownTaleOfDecisionFrameworks}
</DecisionMakingFrameworks>.

Select the top 20 frameworks with the highest value (1-10) for RISK ANALYSIS and 
return them in a Markdown table format in descending order.

Dot not provide any other text or explanation.  
Return the Markdown table with the top 20 RISK ANALYSIS frameworks without any tick marks (```) or other text.
Re-number the frameworks in the table in ascending order starting from 1.
""";

var riskAnalysisFrameworksPromptResponse = await semanticKernel.InvokePromptAsync(riskAnalysisFrameworksPrompt);
// This will append the new markdown table to the previous markdown tables
var riskAnalysisFrameworks = riskAnalysisFrameworksPromptResponse.GetValue<string>();

// Write the final count and markdown table to the Console
Console.WriteLine(riskAnalysisFrameworks);

Finished Gethering Intelligence Iteration: 1
Finished Gethering Intelligence Iteration: 2
Finished Gethering Intelligence Iteration: 3
Finished Gethering Intelligence Iteration: 4
Finished Gethering Intelligence Iteration: 5

Filtering for RISK ANALYSIS frameworks...
| Number | Decision-Framework Name                                | Value | Description                                                                                                                                                                             |
|--------|--------------------------------------------------------|-------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 1      | Rational Decision-Making Model                         | 9     | Follows logical steps to identify, evaluate, and choose the best alternative based on systematic criteria.                          

| Number | Decision-Framework Name                                | Value | Description                                                                                                                                                                             |
|--------|--------------------------------------------------------|-------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 1      | Rational Decision-Making Model                         | 9     | Follows logical steps to identify, evaluate, and choose the best alternative based on systematic criteria.                                                                               |
| 2      | Multi-Criteria Decision Analysis (MCDA)                | 9     | Evaluates options against multiple weighted criteria, ideal for complex decisions with varied performance measures.                                                                      |
| 3      | Analytic Hierarchy Process (AHP)                       | 9     | Breaks down complex decisions into a hierarchy of simpler sub-decisions and applies pairwise comparisons to quantify their relative importance, enhancing structured analysis.      |
| 4      | Bayesian Decision Theory                               | 9     | Integrates prior knowledge with new evidence using Bayes' theorem, enabling probabilistic updates and adaptive decision-making under uncertainty.                                        |
| 5      | Systems Thinking                                       | 9     | Emphasizes understanding and addressing interdependencies within a whole system, facilitating holistic, long-term solutions over isolated fixes.                                        |
| 6      | Real Options Analysis                                  | 9     | Uses financial option theory to evaluate strategic investments by quantifying the value of flexibility under uncertainty, making it ideal for incremental decision-making.           |
| 7      | Expected Utility Theory                                | 9     | Maximizes decision quality under uncertainty by calculating the expected outcomes weighted by their utilities, providing a rigorous quantitative basis for choice.                   |
| 8      | Cost-Benefit Analysis                                  | 8     | Quantifies and compares the costs and benefits of different options to determine the net value of decisions.                                                                             |
| 9      | Decision Trees                                         | 8     | Visual maps that outline decision paths, including possible outcomes, probabilities, and consequences for complex decision processes.                                                    |
| 10     | OODA Loop                                              | 8     | A dynamic cyclical framework (Observe, Orient, Decide, Act) that accelerates decision-making in rapidly changing environments.                                                           |
| 11     | Six Thinking Hats                                      | 8     | Encourages parallel thinking with distinct perspectives (rational, emotional, creative, etc.) to ensure balanced evaluation of options.                                                  |
| 12     | Scenario Planning                                      | 8     | Explores diverse future scenarios to prepare strategies that are robust against uncertainty and external disruptions.                                                                     |
| 13     | Failure Modes and Effects Analysis (FMEA)              | 8     | Identifies potential failure points and assesses their impact to proactively mitigate risks and improve decision quality.                                                                  |
| 14     | PDCA Cycle                                             | 8     | Implements an iterative loop of planning, doing, checking, and acting for continuous improvement in processes and decisions.                                                              |
| 15     | Root Cause Analysis (Five Whys)                        | 8     | Uses iterative questioning to trace a problem back to its underlying cause, ensuring more effective problem-solving.                                                                      |
| 16     | Delphi Method                                          | 8     | Aggregates expert opinions through iterative surveys to reach consensus, particularly useful for decisions in uncertain or complex fields.                                               |
| 17     | PESTLE Analysis                                        | 8     | Examines Political, Economic, Social, Technological, Legal, and Environmental factors to offer a comprehensive view of the external influences on a decision.                         |
| 18     | Cynefin Framework                                      | 8     | Provides a sense-making model that categorizes problems into simple, complicated, complex, and chaotic domains to select the most appropriate decision-making approach.                |
| 19     | Game Theory Analysis                                   | 8     | Applies strategic interaction principles to assess and predict the behavior of competing decision-makers, crucial in negotiating and competitive environments.                        |
| 20     | Premortem Analysis                                     | 8     | Anticipates potential failures by imagining a decision's failure in advance and then working backward to identify possible causes, enhancing risk mitigation strategies.             |

### Step 3 - Gathering Intelligence with External Knowledge Stores